In [1]:
from bs4 import BeautifulSoup
import requests
import time
import csv
import json
import pandas as pd
import sys

In [2]:
df = pd.read_excel('C:/Users/e0267625/Desktop/links.xlsx')

In [3]:
articles = 0
lost_articles = 0

In [ ]:
file = open('article.json', 'a+')
for i in range(len(df['Links'])):
    url = df['Links'][i]
    response = requests.get(url)
    response.encoding = 'utf-8'
    time.sleep(2)
    soup = BeautifulSoup(response.text, 'html.parser')
    if soup.select('time')[0].get_text():
        date = soup.select('time')[0].get_text()
    else:
        lost_articles = lost_articles + 1
        continue
    if soup.select('span.readingTime')[0]['title']:
        read_time = soup.select('span.readingTime')[0]['title']
    else:
        lost_articles = lost_articles + 1
        continue
    if soup.select('button.js-multirecommendCountButton'): 
        shares = soup.select('button.js-multirecommendCountButton')[0].get_text()
    else:
        lost_articles = lost_articles + 1
        continue
    if soup.select('.ds-link')[0].get_text():
        author = soup.select('.ds-link')[0].get_text()
    else:
        lost_articles = lost_articles + 1
        continue
    if soup.select('.graf--p'):
        contents = soup.select('.graf--p')
        articles = articles + 1
    else:
        lost_articles = lost_articles + 1
        continue
    content = ''
    for f in contents:
        content += f.get_text().replace('\u200a','') 
    data = [{'category':df["Topic"][i],'title':df["Titles"][i],'author':author,'date':date,'read_time':read_time,'shares':shares,'content':content}]
    data = json.dumps(data) 
    sys.stdout.write("Articles scrapped: %d | Articles lost: %d  \r" % (articles,lost_articles) )
    sys.stdout.flush()
    file.write(data)
    file.write('\n')
file.close()